<a href="https://colab.research.google.com/github/GiuniorCandido/MAC0110/blob/master/EP10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo

Manipular listas em um exemplo prático. Empregando funções com listas e com ajuda de algumas operações básicas seu programa será usado para sintetizar o som de guitarra usando o algoritmo Karplus-Strong. Esse algoritmo desempenhou um papel seminal no surgimento da síntese por modelagem física em que uma descrição física de um instrumento musical é usada para sintetizar o seu som eletronicamente.

Este EP é uma adaptação de Guitar Hero desenvolvido por Andrew Appel, Jeff Bernstein, Maia Ginsburg, Ken Steiglitz, Ge Wang, e Kevin Wayne da Universidade de Princeton e Guitar Heroine desenvolvida por Varun Pai e John DeNero da Universidade da California em Berkeley.

Descrição

Neste EP, usaremos o que aprendemos até agora sobre listas para usando nossos computadores simularmos o tocar de uma guitarra simples. Após fazer as funções pedidas, você terá transformado seu computador em um instrumento musical simples totalmente funcional, capaz de tocar algumas notas, acordes e músicas.

Como já mencionamos, este EP é uma adaptação de Guitar Heroine desenvolvida em UC em Berkeley por Varun Pai e John DeNero. No texto deste EP há imagens e textos adaptadas do Guitar Heroine. Em particular, utilizaremos o programa server.py desenvolvido por Varun Pai e John DeNero. Este programa sintetiza, ou seja, reproduz eletronicamente, o som de uma guitarra simples. O programa pode ser baixado daqui.

Antes de iniciar, vamos revisar alguns conceitos que serão úteis ao longo do desenvolvimento desse EP.

Sons no computador
No final desse texto, em um apêndice, há uma descrição um pouco mais longa, mas não muito, sobre Sons no computador. A leitura deste apêndice é desnecessária.

Onda Sonora: é um sinal analógico, o que significa que, com o tempo, ela é composta de infinitos valores. Obviamente, computadores têm espaço limitado e, portanto, não podem armazenar um número infinito de valores na memória. Para representar uma onda sonora usando um computador, nós a amostramos em intervalos específicos. Se você pensar em uma onda como uma função contínua f(t), então amostragem significa simplesmente escolher determinados valores para t para calcular f(t) e armazenar os valores resultantes em uma lista. Portanto, para um computador, uma onda sonora é apenas uma longa lista de números, onde cada número é uma amostra da onda em um determinado instante.

Taxa de amostragem: é o número de vezes em que realizamos a amostragem em uma unidade de tempo. Para áudio, esse valor é tipicamente de 44100 hertz, que também é escrito como 44.1 kHz. Isso significa que 44100 valores da função são calculados ou coletados a cada segundo. Esse valores são obtidos em intervalos regulares de tempo e armazenados em uma lista. Portanto, sabemos agora como o som é em geral representado, mas onde aqui entram as notas musicais? Cada nota na escala cromática moderna está associada a uma certa frequência.

Frequência: a frequência de uma nota corresponde ao número de períodos da onda sonora dessa nota que ocorre a cada segundo. Por exemplo, a 49ª tecla de um piano moderno, a nota concerto A ou Lá 440, tem uma frequência de 440 Hz.

Resumo: sons são representados no computador por uma lista de números que são amostragens em uma determinada frequência.

Algoritmo Karplus-Strong

Agora que temos uma ideia física do que está acontecendo ao tocar uma corda de guitarra, como podemos modelá-la com um computador? O algoritmo de Kevin Karplus e Alex Strong é um método de síntese de modelagem física que para o som do dedilhar de uma corda de guitarra ou de alguns tipos de percussão.

Ao dedilharmos uma corda de guitarra, inicialmente, a corda é altamente energizada e vibra rapidamente, criando um som bastante complexo, ou seja, rico em harmônicos. Gradualmente, devido ao atrito entre o ar e a corda, a energia da corda vai se decipando e a onda se torna menos complexa, resultando em um tom "mais puro" com menos harmônicos. Depois de algum tempo, toda a energia se esvai e a corda pára de vibrar.

Como funciona?

Ao iniciarmos a dedilhar uma corda de guitarra ela pode conter energia em qualquer frequência. Essa energia é um ruído, representado por uma lista com valores aleatórios entre -1/2 e +1/2.

Em seguida, depois que a corda é dedilhata, ela vibra. Essa vibração causa um deslocamento de ar que se espalha em ondas ao longo do tempo. O algoritmo de Karplus-Strong simula essa vibração mantendo uma lista das amostras: o algoritmo "exclui" repetidamente a primeira amostra da lista e adiciona ao da final da lista a média da amostra "excluída" e da amostra seguinte, multiplicada por um fator de decaimento de energia de 0.996. O "exclui" está entre aspas pois não removeremos a amostra da lista, mas deixaremos de considerá-la nos próximos passos. O fator de decaimento modela a ligeira dissipação de energia à medida que a onda percorre a corda da guitarra de um extremo ao outro, indo e voltando.

A atualização Karplus-Strong pode ser definida em termos das amostras anteriores, segundo a seguinte equação:

s_t = 0.996 * \frac{s_{t-p} + s_{t-p+1}}{2}

Na equação acima, s é uma sequência de amostras e p depende da frequência da nota que queremos simular, por exemplo, para A temos que p depende da frequência de 440 Hz. Para gerar amostras para uma nota, começaremos com p amostras de ruído e adicionamos muitas vezes uma nova amostra usando a atualização Karplus-Strong. A seguir, um passo da atualização é ilustrada para p = 10.

Hmm, em linhas bem superficiais, os dois componentes principais que fazem o algoritmo Karplus-Strong funcionar são:

o mecanismo de feedback: O comprimento p da atualização determina a frequência fundamental do som resultante. Sonicamente, o mecanismo de feedback reforça apenas a frequência fundamental e seus harmônicos (frequências em múltiplos inteiros da fundamental). O fator de decaimento de energia, 0,996 neste caso, modela a leve dissipação de energia à medida que a onda faz uma ida e volta através da corda.

a operação de média: Esta operação serve como um filtro passa-baixo que remove frequências mais altas enquanto permite frequências mais baixas passem, daí o nome. Como está no caminho do feedback, isso tem o efeito de atenuar gradualmente os harmônicos mais altos, enquanto mantém os mais baixos, o que é semelhante à maneira como realmente as cordas tocadas soam.

Uma vez obtida a simulação de uma nota musical podemos criar música com os acordes necessários.

Acordes

Na música, os acordes são sons obtidos tocando várias notas simultaneamente. Felizmente, os acordes são fáceis de implementar quando temos as amostras que representam os sons notas. Um acorde é a superposição de suas notas componentes. Digamos que queremos construir um acorde C maior que representaremis pela lista ['C', 'E', G']. Cada uma dessas três notas tem seu som representado por amostras. Para construir o acorde precisamos fazer uma adição elementar das amostras de cada nota. Por exemplo, se as amostras de 'C' fossem [0.25, 0.4, 0.3, ...],
as amostras de 'E' foram [0.25, 0.2, 0.1, ...] e as amostras de 'G' foram [0.15, 0.1, 0.35, ...], então
as amostras do acorde C maior seriam [0.65, 0.7, 0.75, ...].



O que você deve fazer

Você deve completar o programa no arquivo ep10.py. Esse programa contém:

funções que você deve fazer: indice(), crie_ruido(), superposicao(), aplique_ks()
funções que você deve consertar: crie_guitarra(), crie_musica()
funções totalmente feitas e que não devem ser alteradas: crie_som_nota() e 
crie_som_acorde():

No início do programa há uma constante de nome TESTANDO. Sugerimos que enquanto você estiver fazendo as suas funções deixe o valor de TESTANDO como True. Quando for escutar os sons, faça TESTANDO passar a ser False.

Função indice()

A função indice() recebe um valor item e uma lista de lst e retorna o índice da posição de lst em que é igual a item. Se nenhuma posição de lst é igual a item a função imprime uma mensagem "SOCORRO! Não achei item" e retorna None. Se mais de uma posição de lst é igual a item a função retorna o maior índice.


Exemplos
    
    In [7]: lst = ['A', 1, 2.3, True, 1]
    In [8]: indice('A', lst)
    Out[8]: 0
    In [9]: indice(1, lst)
    Out[9]: 4
    In [10]: indice(False, lst)
    In [11]: i = indice(False, lst)
    SOCORRO! Não achei False
    In [12]: i == None
    Out[12]: True      

Função crie_ruido()
A função crie_ruido() recebe um inteiro n e retorna uma lista com n valores aleatórios entre -0,5 e 0,5. Para criar a lista use a chamada da função random.uniform(-0.5, 0.5) que retorna um número aleatório em ponto flutuante entre -0.5 e 0.5.

Exemplos
    
    In [13]: import random
    In [14]: random.seed(0) # para reprodutibilidade
    In [15]: crie_ruido(3)
    Out[15]: [0.3444218515250481, 0.2579544029403025, -0.079428419169155]
    In [16]: random.seed(1) # para reprodutibilidade
    In [17]: lst = crie_ruido(2)
    In [18]: lst
    Out[18]: [-0.3656357558875988, 0.3474337369372327]

Função superposicao()

A função superposicao() recebe duas lista de números som1 e som2 de mesmo comprimento. A função cria e retorna a lista resultante da adição dos componentes das duas listas, componente a componente.

Exemplo
    
    In [22]: random.seed(1) # para reprodutibilidade
    In [23]: som1 = crie_ruido(3)
    In [24]: som2 = crie_ruido(3)
    In [25]: som1
    Out[25]: [-0.3656357558875988, 0.3474337369372327, 0.26377461897661403]
    In [26]: som2
    Out[26]: [-0.2449309742605783, -0.004564912908059049, -0.050508935211261874]
    In [27]: som = superposicao(som1, som2)
    In [28]: som
    Out[28]: [-0.6105667301481771, 0.3428688240291736, 0.21326568376535215]

Função aplique_ks()

A função aplique_ks() recebe uma lista ruido com pelo menos 2 elementos e um inteiro n. A função cria e retorna uma lista som que respeita as seguintes condições:

se p é o comprimento da lista ruido, então o comprimento da lista som é p+n,
os primeiros p componentes de som são iguais aos correspondentes de ruido,
cada um dos demais componentes da lista som deve ser obtido através da equação do algoritmo de Karplus-Strong.

Exemplos
    
    In [15]: ruido = [.2, .4, .5]
    In [16]: som = aplique_ks(ruido, 1) 
    In [17]: som
    Out[17]: [0.2, 0.4, 0.5, 0.29880000000000007]
    In [18]: som = aplique_ks(ruido, 2) 
    In [19]: som
    Out[19]: [0.2, 0.4, 0.5, 0.29880000000000007, 0.4482]
    In [20]: som = aplique_ks(ruido, 3)      
    In [21]: som
    Out[21]: [0.2, 0.4, 0.5, 0.29880000000000007, 0.4482, 0.39780240000000006]
    In [22]: 0.996*(0.5+0.2988)/2
    Out[22]: 0.3978024
    In [23]: ruido
    Out[23]: [0.2, 0.4, 0.5]

Função crie_guitarra()

A tabulação da função crie_guitarra() foi removida. Portanto, antes de utilizá-la, você deve corrigir essa tabulação. Não altere nada além da tabulação dessa função.

A função crie_guitarra() recebe uma lista teclas e uma lista notas de mesmo comprimento. Cada posição da lista teclas contém uma string associada a uma posição do teclado do computador. Cada posição da lista notas contém uma string associada a uma nota musical. A função cria e retorna uma lista guitarra representando uma guitarra. Cada posição guitarra[i] é uma lista de 3 posições:

guitarra[i][0] é teclas[i]

guitarra[i][1] é notas[i]

guitarra[i][2] é uma lista som com os números que sintetizam o som associado a notas[i]

Exemplo
Para o teste a seguir certifique-se que a variável TESTANDO está com True.

    In [5]: random.seed(123) 
    In [6]: guitarra = crie_guitarra()
    In [7]: guitarra
    Out[7]: 
    [['a', 'C', [-229, -211, -47, -200, 205, -236, -219, -128, -123, 2]],
     ['w', 'C#', [18, -85, 180, -174, -83, -85, -33, 46, 2, -128]],
     ['s', 'D', [-130, -255, -32, -211, 49, -220, -192, -143, -121, -80]]]

Função crie_musica()

A tabulação da função crie_musica() foi removida e algumas linhas trocadas. Portanto, antes de utilizá-la, você deverá corrigir essa tabulação e a ordem dos comandos.

A função crie_musica() recebe uma lista guitarra criada pela função crie_guitarra() e que representa uma guitarra. Recebe ainda uma lista partitura em que cada componente é:

uma string representado uma nota musical, como por exemplo, 'A','C#',...
uma lista de strings representado um acorde, como por exemplo ['A', 'C#', 'E']

A função cria e retorna uma lista musica. Cada posição musica[i] corresponde ao som (=lista de números) associada a uma nota musical ou um acorde em partitura[i]. Portanto, o comprimento da lista musica é igual ao da lista partitura.
ExemplosPara os teste a seguir certifique-se que a variável TESTANDO está com True.

    In [2]: guitarra = crie_guitarra() 
    In [3]: guitarra
    Out[3]: 
    [['a', 'C', [-231, -42, 46, 49, 224, -203, -136, 1, 47, 136]],
    ['w', 'C#', [108, 0, 138, -29, -76, 69, 54, 68, 53, -52]],
    ['s', 'D', [-126, -230, 44, -222, 77, -27, -178, -92, -88, -72]]]
    In [4]: partitura = ['C'] 
    In [5]: musica = crie_musica(guitarra, partitura) 
    
    In [6]:      musica                                                                              
    Out[6]: [[-231, -42, 46, 49, 224, -203, -136, 1, 47, 136]]
    In [7]: partitura = ['C','C'] 
    In [8]: musica = crie_musica(guitarra, partitura) 
    In [9]: musica
    Out[9]: 
    [[-231, -42, 46, 49, 224, -203, -136, 1, 47, 136],
    [-231, -42, 46, 49, 224, -203, -136, 1, 47, 136]]
    In [10]: partitura = ['C','C',['C#','D']] # ['C#','D'] é um acorde] 
    In [11]: musica = crie_musica(guitarra, partitura)   
    In [12]: musica
    Out[12]: 
    [[-231, -42, 46, 49, 224, -203, -136, 1, 47, 136],
    [-231, -42, 46, 49, 224, -203, -136, 1, 47, 136],
    [-18, -230, 182, -251, 1, 42, -124, -24, -35, -124]]

Ouvir sons

Depois de fazer as funções indice(), crie_ruido(), superposicao(), aplique_ks() e consertar a tabulação da função crie_guitarra() e crie_musica() podemos escutar um pouco de som. Antes de continuar atribua False a variável TESTANDO.

Vá na função produza_musica() e escolha/descomente uma partitura ou faça a sua. Baixe o programa server.py daqui. Execute esse programa server.py no Spyder. Você deverá ver a mensagem

    Vá para o seu navegador em http://localhost:8000/ e cruze os dedos ;-)
    Para encerrar venha até está janela e tecle CTRL+C
Abra o seu navegador e vá para http://localhost:8000. Se tudo der certo, você verá letras na tela seguidas de nomes de notas musicais. Pressione qualquer letra listada no teclado para reproduzir o som nota correspondente. Isso foi testado com o Firefox e Google Chrome, não sabemos o que acontece com os outros navegadores.

Nota: Os computadores têm controles de volume, portanto, se você não ouvir nenhum som, tente aumentar o volume. Para alterar seu programa, volte para o Python Shell onde o programa server.py está sendo executado e tecle CRTL+C. Altere seu programa e execute o programa server.py novamente. Depois de brincar um pouco, volte para o Python Shell onde o programa server.py está sendo executado e tecle CRTL+C.

Suas músicas

Agora temos todos as funções necessárias para fazer algumas músicas! Você pode se divertir modificando ou criando uma lista partitura na função produza_musica(). Essa partitura é uma lista de strings representando notas, como A, F# ou acordes representados por uma lista de notas como ['C', 'E', G']. A lista que você produzir são as notas que serão sequencialmente sintetizadas.

Depois de ter inserido uma música, salve seu ep10.py e mais uma vez execute o server.py a Python Shell e no seu navegador vá até http://localhost:8000. Agora aperte o botão Ouvir música e aproveite seu trabalho!

Parabéns! Você conseguiu transformar seu computador em uma guitarra simples.



Roteiro

Baixe o arquivo ep10.py para uma pasta do seu computador.

Baixe o arquivo server.py daqui para a mesma pasta do seu computador em que está ep10.py. O único arquivo que deverá ser depositado nesta página é ep10.py.

Implemente e teste a função indice().

Implemente e teste a função crie_ruido().

Implemente e teste a função superposicao().

Implemente e teste a função aplique_ks().

Conserte a tabulação da função crie_guitarra().

Conserte a tabulação e a ordem dos comandos crie_musica().

Neste ponto seu EP está completo e se desejar pode depositá-lo nesta página. Verifique as mensagem do programa avaliador. Se necessário faça eventuais correções e ressubmeta o seu EP.

Para se divertir e escutar sons que seu ep10.py produz, execute o programa server.py.

Para criar músicas, e se divertir mais ainda, crie uma nova lista partitura na função produza_musica().

Para se divertir mais mais ainda e criar músicas mais interessantes, implemente mais notas musicais, mas a versão do seu EP que deve ser entregue é a simples.

Há muitas maneiras para você melhorar e brincar com este EP!

In [1]:
#####################################################################
# MÓDULOS A SEREM UTILIZADOS NO PROGRAMA
import random

#####################################################################
# CONSTANTES

# enquanto estiver testando o seu programa deixe True
TESTANDO = True

# nunca se sabe ...
SOCORRO  =  False

# indica o volume do som, 512 parece ok.
# qualquer coisa, ajuste o volume do seu computador
VOLUME = 512 # 256

#-------------------------------------------------------------------------
# NOSSO ALBUM DE MÚSICAS, mais ou menos
# MUSICA, o mais perto que consegui, faltam notas musicais...,
# Hmm, vocês podem expandir..., mas não entregue a versão expandida.
STAR_WAY_TO_HEAVEN = [
    'C', 'E', 'A', 'B', 'E', 'C',
    'B', 'C+', 'F', 'C', 'C+', 'F#', 'D',
    'A', 'F', 'E', 'C', 'A', 'C', 'E', 'C', 'A'
]

FRERE_JACQUES = [
    'C', 'D', 'E', 'C',
    'C', 'D', 'E', 'C',
    'E',  'F', 'G',
    'E',  'F', 'G',
    'G',  'A', 'G', 'F', 'E', 'C',
    'G',  'A', 'G', 'F', 'E', 'C',
    'C',  'G', 'C',  # hmm, esse sol devia ser uma oitava abaixo, fui mal...
    'C',  'G', 'C' 
]

AQUARELA = [
    'G',   'G', 'C+', 'C+', 'B', 'A', 'G', 'G',
    'C+', 'C+', 'B',   'A', 'G', 'G', 'A', 'A',
    'G',   'G', 'C+', 'C+', 'B', 'A', 'G', 'G',
    'C+', 'C+', 'B',   'A', 'G', 'G', 'A', 'G', 'F'
]    

# ---------------------------------------------------------------
# ESCALA MAIOR
# Tabela mostrando a correspondência entre teclas do computador, notas musicais e frequencias:
# a TECLA[i] corresponde a nota musical NOTA[i] que tem frequencia (aprx) FREQUENCIA[i]
# Hmm, vocês podem expandir as notas,..., mas não entregue a versão expandida.
if TESTANDO:
    # 44100 é a taxa de amostragem "padrão" de som digital
    TAXA_DE_AMOSTRAGEM = 12
    #     
    NO_AMOSTRAS = 10 
    # não tem nada a ver com som, apenas para testes numéricos
    TECLAS      = [  'a',  'w',  's']
    NOTAS       = [  'C', 'C#',  'D']
    FREQUENCIAS = [    2,    2,    2]
else:
    # 44100 é a taxa de amostragem "padrão" de som digital
    TAXA_DE_AMOSTRAGEM = 44100

    # Hmm, esse 20000 é chutado...
    # Quanto maior, mais 'longo' é o som, quanto menor, mais 'curto'
    # Hmm, quanto maior, maiores são as listas e mais tempo para carregar no browser
    NO_AMOSTRAS = 20000 # 30000 
    # para executar server.py e sintetizar som, TESTANDO deve ser False
    # notas musicais, C+ é invensão para C "um oitava acima"
    #            dó   dó#   ré    ré#    mi    fá    fá#   sol  sol#    lá   lá#    si    dó  (uma oitava acima)
    NOTAS  = [  'C', 'C#',  'D', 'D#',  'E',  'F',  'F#',  'G', 'G#',  'A', 'A#',  'B',  'C+']
    #            dó   dó#   ré    ré#    mi    fá   fá#   sol  sol#   lá    lá#    si    dó  (uma oitava acima)
    TECLAS = [  'a',  'w',  's',  'e',  'd',  'f',  't',  'g',  'y',  'h',  'u',  'j',   'k']
    # frequencias
    FREQUENCIAS = [
        523.25,  # dó, 
        554.36,  # dó sustenido,
        587.33,  # ré, 
        622.25,  # ré sustenido, 
        659.26,  # mi, 
        698.46,  # fá, 
        739.99,  # fá sustenido, 
        783.99,  # sol, 
        830.61,  # sol sustenido, 
        880.00,  # lá 
        932.33,  # lá sustenido
        987.77,  # si 
        1046.50  # dó uma oitava acima, sol maior
    ]

# indices da tabela que representa a guitarra
IND_TECLA = 0  # guitarra[i][0] representa uma tecla (str)
IND_NOTA  = 1  # guitarra[i][1] representa uma nota musica (str)
IND_SOM   = 2  # guitarra[i][2] representa o som da nota (list)

#####################################################################
#
# Funções a serem escritas ou consertadas.
#
#-------------------------------------------------------------------
def indice(item, lst):
    '''(obj, list) -> int ou None

    RECEBE um objeto  `item` e uma lista de `lst`.
    RETORNA o índice da posição de `lst` em que é igual a `item`.

    Se nenhuma posição de `lst` é igual a item a função IMPRIME uma 
    mensagem "SOCORRO! Não achei item" e RETORNA None.

    Se mais de uma posição de `lst` é igual a item a função RETORNA o
    maior índice.
    '''
    # modifique o código abaixo para conter a sua solução.
    var1 = 0
    krav_maga = False
    var2 = len(lst)
    for i in range(var2):
    	if item == lst[i]:
    		var1 = i
    		krav_maga = True
    if krav_maga:
    	return var1
    if not krav_maga:
    	print('SOCORRO! Não achei item')
    	return None
     
#-------------------------------------------------------------------
def crie_ruido(n):
    '''(int) -> list

    RECEBE um inteiro n.
    RETORNA uma lista com n valores aleatórios entre -0,5 e 0,5. 

    Para criar a lista use a chamada da função random.uniform(-0.5, 0.5) 
    que retorna um número aleatório em ponto flutuante entre -0.5 e 0.5.
    '''
    # modifique o código abaixo para conter a sua solução.
    var1 = 1
    lst1 = []
    while var1 <= n:
    	lst1 += [random.uniform(-0.5, 0.5)]
    	var1 += 1
    return lst1

#-------------------------------------------------------------------
def superposicao(som1, som2):
    '''(list, list) -> list

    RECEBE duas lista de números `som1` e `som2` de  MESMO comprimento.
    A função CRIA e RETORNA a lista resultante da adição
    dos componentes das duas listas, componente a componente.
    '''
    # modifique o código abaixo para conter a sua solução.
    lst1 = []
    var1 = len(som1)
    for i in range(var1):
    	var2 = som1[i] + som2[i]
    	lst1 += [var2]
    return lst1
    
#--------------------------------------------------------------------
def aplique_ks(ruido, n):
    '''(list, int) -> list

    RECEBE uma lista `ruido` e um inteiro `n`.
    A função CRIA e RETORNA uma lista `som` que respeita as seguintes condições:

      * se `p` é o comprimento da lista `ruido`, então o comprimento da lista 
        `som` é `p+n`.
      * os primeiros `p` componentes de `som` são iguais a `ruido`
      * cada um dos demais componentes da lista `som` deve ser obtido através da 
        equação do algoritmo de Karplus-Strong.
        
    PRÉ-CONDIÇÃO: a função supõe que p é pelo menos 2.    
    '''
    #modifique o código abaixo para conter a sua solução.
    p = len(ruido)
    var1 = p + n
    som = var1 * [0]
    for i in range(p):
    	som[i] = ruido[i]
    for i in range(p, p + n):
    	som[i] += 0.996 * ((som[i - p] + som[i - p + 1]) / 2)
    return som

print(aplique_ks([.2, .4, .5], 3))

####################################################################
#
# As funções crie_guitarra() e crie_musica() devem ser consertadas
#    
####################################################################

#--------------------------------------------------------------------    
def crie_guitarra(teclas=TECLAS, notas=NOTAS):
    '''(list, list) -> list[3]

    RECEBE uma lista `teclas` e uma lista `notas` de mesmo comprimento.
    Cada posição da lista `teclas` contém uma string associada a uma posição do 
    teclado do computador.
    Cada posição da lista `notas` contém uma string associada a uma nota musical.

    A função CRIA e RETORNA uma lista `guitarra` represetando uma guitarra.
    Cada posição `guitarra[i]` é uma lista de 3 posições 
 
       * `guitarra[i][0] é `teclas[i]
       * `guitarra[i][1] é `notas[i]`
       * `guitarra[i][2] é  uma lista `som` com os números que sintetizam o 
                         som associado a `notas[i]`

    DEPENDE da função crie_som_nota() que está TOTALMENTE feita.
    '''
    guitarra = []
    n = len(teclas)
    for i in range(n):
    	nota = notas[i]
    	tecla = teclas[i]
    	som_nota = crie_som_nota(nota)
    	guitarra += [[tecla, nota, som_nota]]
    return guitarra

#--------------------------------------------------------------------
def crie_musica(guitarra, partitura):
    ''' (list[3], list) -> list

    RECEBE uma lista `guitarra` criada pela função `crie_guitarra()` e que
    representa uma guitarra.
    RECEBE ainda uma lista `partitura` em que cada componete é:

        * uma string representado uma nota musical, como por exemplo, 'A','C#',...
        * uma lista de strings representado um acorde, como por exemplo ['A', 'C#', 'E']

    A função CRIA e RETORNA uma lista `musica`.
    Cada posição `musica[i]` corresponde ao som (lista de números) associada a 
    nota ou acorde em partitura[i]. Portanto, o comprimento da lista `musica`
    é igual ao da lista `partitura`.

    DEPENDE da função crie_som_acorde() que está TOTALMENTE feita.
    '''
    n = len(partitura)
    musica = [0]*n
    for i in range(n):
    	acorde = partitura[i] # srt para nota ou list para acorde
    	som_acorde = crie_som_acorde(guitarra, acorde) # list de números
    	musica[i] = som_acorde 
    return musica


####################################################################
#
# A função produza_musica() é apenas para você se divertir
#    
####################################################################

#-------------------------------------------------------------------
def produza_musica():
    '''(None) -> list

    RETORNA uma lista `musica` criada pela função `crie_musica()` e 
    correspondente a música na lista `partitura` que é uma
    variável local.

    Cada posição da lista `partitura` é uma string correspondente 
    a uma nota musical como `A`, `A#`, `D`,... ou uma lista de
    strings correspondente a uma acorde como ['A', 'C#', 'E'].

    Essa você é apenas para você se divertir criando suas próprias músicas
    e escutando-as através do programa `server.py`.

    Este exemplo começa com a canção "Brilha, brilha".

    DEPENDE das função crie_guitarra() e crie_musica() que você consertou
    '''
    # 1 crie uma guitarra
    guitarra  = crie_guitarra()
    
    # exemplo que toca apenas uma nota
    # musica = [crie_som_nota('A')] 
    # return musica

    # 2 crie uma partitura
    # partitura = ['F', 'G', 'A', 'F', 'G',  'A', ['B', 'D', 'F']]
    # partitura = ['A', 'B','C#', 'A', 'B', 'C#', ['B', 'D', 'F']]
    partitura = FRERE_JACQUES
    # partitura = STAR_WAY_TO_HEAVEN
    # partitura = AQUARELA
    
    # 3. crie a música associada partitura 
    musica = crie_musica(guitarra, partitura)
    
    # 4. retorne a música
    return musica 


####################################################################
#
# Não altere nada deste ponto em diante
#    
####################################################################

#-------------------------------------------------------------------
def crie_som_nota(nota, n=NO_AMOSTRAS, taxa_de_amostragem=TAXA_DE_AMOSTRAGEM, volume=VOLUME):
    '''(str, int, int, int) -> list

    RECEBE:

       - uma string `nota` reprentando uma no musical
       - um inteiro `n` que será o número de amostras e tamanho da lista que será criada e 
         retornada
       - a taxa_de_amostragem que é o número de amostras por segundo, tiicamente 44110
       - um inteiro que indica o volume do som que será criado

    RETORNA uma lista `som` de com `n` números que será usado para sintetizar o 
    som correspondente a `nota` dada.

    DEPENDE das funções crie_ruido() e aplique_ks().
    '''
    # 1. pegue a frequência da nota
    ind_nota   = indice(nota, NOTAS)
    frequencia = FREQUENCIAS[ind_nota]
    if SOCORRO:
        print("---------------------------------------")
        print("crie_som_nota(): frequencia de '%s'= %f"%(nota, frequencia))
        
    # 2. calcule a atraso, aqui é o UNICO LUGAR que usa a frequência da nota
    atraso   = int(taxa_de_amostragem / frequencia)
    if SOCORRO:
        print("---------------------------------------")
        print("crie_som_nota(): atraso=", atraso)
        
    # 3. crie o ruido: o comprimento `atraso` do ruído é função da frequência da nota
    ruido = crie_ruido(atraso)
    if SOCORRO:
        print("---------------------------------------")
        # print("ruido=", ruido) # lista longa de valores entre -0.5 e 0.5
        print("crie_som_nota(): len(ruido)=", len(ruido))

    # obtenha o som da corda estendendo o ruído com o algoritmo de Karplus-Strong
    amostras = aplique_ks(ruido, n - atraso)
    if SOCORRO:
        print("---------------------------------------")
        # print("amostras depois de ks=", amostras) # lista muito longa de floats
        print("crie_som_nota(): len(amostras)=", len(amostras))

    # ajuste o volume e do som para ser apresentado por server.py
    som = []
    for i in range(len(amostras)):
        som += [int(volume*amostras[i])]
    if SOCORRO:    
        print("---------------------------------------")
        # print("som =", som) # lista muito longa de inteiros
        print("crie_som_nota(): len(som) =", len(som))
        
    return som

#-----------------------------------------------------------------------------
def crie_som_acorde(guitarra, notas):
    ''' (list[3], str ou list) -> list

    RECEBE uma lista `guitarra` criada pela função `crie_guitarra()` e que representa
    uma guitarra.
    RECEBE ainda uma string ou lista `notas`. Se `notas` é uma string então corresponde
    a uma nota musical. Se `notas` é uma lista então é uma lista de strings que
    representam um acorde.

    A função CRIA e RETORNA uma lista `som_acorde` que representa o som da nota ou do 
    acorde musical `notas`, dependendo do caso. O som de um acorde é obtido através
    da superposição das notas que o compoem,

    DEPENDE das funções crie_som_nota() e superposicao().
    '''
    if type(notas) == str:
        # na verdade é apenas uma nota e não um acorde
        som_acorde = crie_som_nota(notas)
    else:
        # notas é uma lista de notas
        # crie uma lista para o som do acorde
        som_acorde = [0]*NO_AMOSTRAS
        # pegue o número de notas
        n = len(notas)
        for i in range(n):
            # pegue o indice nota
            k = indice(notas[i], NOTAS)
            # pegue o som da nota na guitarra: é uma lista
            som_nota = guitarra[k][IND_SOM]
            # superponha o _som da nota_ com o som do _acorde atual_,
            # essencialmente 'som_acorde = som_acorde + som_nota
            som_acorde = superposicao(som_acorde, som_nota)
    return som_acorde  

   


[0.2, 0.4, 0.5, 0.29880000000000007, 0.4482, 0.39780240000000006]
